In [533]:
import numpy
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import json
import random
import string
import math
import pandas as pd
import plotly.express as px

Importing the dataset

In [534]:
with open('reviewSelected100.json') as f:
    data = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [535]:
# Create a set of all businesses
business_set = set()
for review in data:
    business_set.add(review['business_id'])
print("Number of businesses: " + str(len(business_set)))

Number of businesses: 153


## Finding the Adjective Phrases of a Randomly Selected Business

Choose a random business - For consistency, we will use the business_id that we generated randomly on the first run (R4R7ttLXfKKWM0VEMoaW4w)

In [536]:
business_set = list(business_set)
selected_business_id = business_set[random.randrange(len(business_set))]

In [537]:
selected_business_id = "R4R7ttLXfKKWM0VEMoaW4w"
selected_business_id

'R4R7ttLXfKKWM0VEMoaW4w'

Extract out the adjective phrases from all the reviews belonging to this business

In [538]:
# Get all the reviews of the business
selected_business_reviews = []
for review in data:
    if review['business_id'] == selected_business_id:
        selected_business_reviews.append(review['text'])

Define the grammar rules for the structure of Adjective Phrases (ADJP), as well as other phrase types. 

In [539]:
# Final
grammar = ('''
    VP: { <TO> <VB> <PRP> <VB.?> | <TO> <VB> <VB.?>? <DT>? <JJ.?>? <NN.?>? }
    NC: { <IN>? ((<PRP.>|<DT>) <NN.?> <JJ.?>? <MD> <VB.?>) | <IN>? ((<DT>|<PRP.>) <JJ.?>? <NN.?> <VB.?> (<NN.?>|<VB.>)) | <IN>? <PRP> <VB.?> (<DT>|<PRP.?>) <NN.?> }
    PP: { (<IN> <DT>? <JJ.?>? <NN.?>) }
    ADJP: { (<IN> <DT>? <NN.?>) | (<PP> <IN> <NN.?>) | <RB.?>* <JJ.?> <PP> | <RB.?>* <JJ.?> <VP> | <RB.?>* <JJ.?> <NC> | <RB.?>* <JJ.?> (<,>? <CC>? <RB.?>* <JJ.?>)* | <JJ.?> (<,>? <CC>? <JJ.?>)* | <JJ.?>* | <RB.?>+ <VB(G|N)> }
    ''')

# Preposition + noun | PP + noun | (adv) adj + PP | (adv) adj + VP | (adv) adj + NC | Adverb Adjective(s) | Adjective chain(s) | Adjective(s) | Some VBG/N

In [540]:
# preprocess review into pos tags for parsing
def preprocess_review_to_pos(text):
    # tokenize the review into sentences
    text = sent_tokenize(text)
    
    # tokenize the sentences into words
    tokens = []
    for sentence in text:
        tokens.append(word_tokenize(sentence))
    
    # generate the pos tags for each tokenized sentence
    results = []
    for lst in tokens:
        results.append(nltk.pos_tag(lst))
        
    return results # output is a list of lists of pos tags

In [541]:
chunkParser = nltk.RegexpParser(grammar)

Parse each review, analysing its phrase structure and extracting out only the adjective phrases

In [542]:
adjective_phrases = []

for review in selected_business_reviews:
    # preprocess the review text
    tagged_tokens = preprocess_review_to_pos(review)
    # parse the review
    for tags in tagged_tokens:
        tree = chunkParser.parse(tags)
        # go through the parse tree and append adjective phrases (ADJP)
        for subtree in tree.subtrees():
            if subtree.label() == "ADJP":
                adjective_phrase = []
                for leaf in subtree.leaves():  # each leaf is a tuple (token, pos_tag)
                    adjective_phrase.append(leaf[0]) # append the token
                adjective_phrase = " ".join(adjective_phrase) # merge the tokens of the phrase
                adjective_phrases.append(adjective_phrase)

In [543]:
adjective_phrases

['nicest',
 'very friendly',
 'various',
 'chill',
 'consistently good',
 'not rotated',
 'very clean',
 'friendly',
 'particular',
 'rewards',
 'Good',
 'frozen',
 'own',
 'officially yogurt',
 'fro-yo',
 'Last',
 'tasty first-time',
 'very friendly',
 'added',
 'inhale',
 'clean and tasty',
 'decent frozen',
 'helpful',
 'vast and plentiful',
 'First',
 'frozen',
 'very nice',
 'Clean and easy',
 'never otherwise thought',
 'first',
 'first',
 'Too exhausted',
 'great',
 'Very white and clean',
 'Enormous',
 'fresh',
 'so happy to be able',
 'friendly',
 'short',
 'more to put',
 'neatest',
 'tiny',
 'own',
 'always great and most',
 'crazy busy',
 'other',
 'really bad for the blonde',
 'great',
 'such',
 'hard',
 'Best',
 'Friendly',
 'always clean',
 'to-go',
 'full',
 'so unprofessional',
 'actually closed',
 'mediocre',
 'big',
 'friendly',
 'fresh',
 'high',
 'hopeful to find a great',
 'froyo',
 'few unfortunate',
 'great',
 'so many',
 'super friendly',
 'always giving',
 'he

We only want the unique adjective phrases that we extracted. To prevent duplication, we will lowercase the adjective phrases and obtain only the unique ones.

In [544]:
# convert to lowercase
for i in range(len(adjective_phrases)):
    adjective_phrases[i] = adjective_phrases[i].lower()

In [545]:
adjective_phrases

['nicest',
 'very friendly',
 'various',
 'chill',
 'consistently good',
 'not rotated',
 'very clean',
 'friendly',
 'particular',
 'rewards',
 'good',
 'frozen',
 'own',
 'officially yogurt',
 'fro-yo',
 'last',
 'tasty first-time',
 'very friendly',
 'added',
 'inhale',
 'clean and tasty',
 'decent frozen',
 'helpful',
 'vast and plentiful',
 'first',
 'frozen',
 'very nice',
 'clean and easy',
 'never otherwise thought',
 'first',
 'first',
 'too exhausted',
 'great',
 'very white and clean',
 'enormous',
 'fresh',
 'so happy to be able',
 'friendly',
 'short',
 'more to put',
 'neatest',
 'tiny',
 'own',
 'always great and most',
 'crazy busy',
 'other',
 'really bad for the blonde',
 'great',
 'such',
 'hard',
 'best',
 'friendly',
 'always clean',
 'to-go',
 'full',
 'so unprofessional',
 'actually closed',
 'mediocre',
 'big',
 'friendly',
 'fresh',
 'high',
 'hopeful to find a great',
 'froyo',
 'few unfortunate',
 'great',
 'so many',
 'super friendly',
 'always giving',
 'he

In [546]:
print("Total number of adjective phrases extracted: " + str(len(adjective_phrases)))
print("Total number of unique adjective phrases: " + str(len(set(adjective_phrases))))

Total number of adjective phrases extracted: 683
Total number of unique adjective phrases: 405


In [547]:
# take only the unique adjective phrases
adjective_phrases = list(set(adjective_phrases))

### Now, we need to determine which adjective phrases are indicative
We perform TF-IDF in order to determine which phrases are unique to the business
- Find the TF of each adjective phrase
- Create a biword/word index to store the Document Frequency of each biword/term
- Find the IDF value of each adjective phrase
- Compute the TF-IDF of each adjective phrase

Adjective phrases can be:
- One word
- More than one word

For one-word phrases, we calculate TF-IDF as per normal, i.e. the TF-IDF value of the word will be the TF-IDF value of the adjective phrase.

For more-than-one-word phrases, we break down the adjective phrase into bigrams (after preprocessing the phrase). From there, we calculate the TF-IDF value of each bigram and then sum them up to obtain the TF-IDF value of the adjective phrase.

First, preprocess the entire dataset before performing TF-IDF

In [548]:
# Sentence tokenizer
def sentence_tokenize(text):
    return sent_tokenize(text)

In [549]:
# Tokenize the sentences -> words
def tokenize(sentences):
    result = []
    for sentence in sentences:
        result.append(word_tokenize(sentence))
    return result

In [550]:
# Lowercase all the text
def convert_to_lowercase(text):
    result = []
    for tokens in text:
        lowercase_tokens = []
        for token in tokens:
            lowercase_tokens.append(token.lower())
        result.append(lowercase_tokens)
    return result

In [551]:
# Remove punctuations except hyphens
def remove_punctuation(sentences):
    punctuation = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'
    result = []
    for sentence in sentences:
        new_sentence = []
        for token in sentence:
            new_token = token.translate(str.maketrans('', '', punctuation))
            if len(new_token) > 0:
                new_sentence.append(new_token)
        if len(new_sentence) > 0:
            result.append(new_sentence)
    return result

In [552]:
# Remove stopwords
def remove_stopwords(sentences):
    result = []
    stopwords = nltk.corpus.stopwords.words('english')
    for sentence in sentences:
        filtered = []
        for word in sentence:
            if word not in stopwords:
                filtered.append(word)
        result.append(filtered)
    return result

In [553]:
# Stemming
def stemming(sentences):
    result = []
    ps = PorterStemmer()
    for sentence in sentences:
        result.append([ps.stem(word) for word in sentence])
    return result

In [554]:
# preprocessing
def preprocess(text):
    text = sentence_tokenize(text)
    text = tokenize(text)
    text = convert_to_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return text
# Outputs a list of lists containing tokens

Create biword/uniword index: Dictionary with bigram/word as the key and the value being the set of businesses that contain that bigram/word, this helps us to retreive document frequency easily.

In [555]:
# Generate bigrams given an array of arrays of strings
def generate_bigrams(sentences):
    bigrams = []
    for sentence in sentences:
        for i in range(len(sentence) - 1):
            bigrams.append((sentence[i], sentence[i+1]))
    return bigrams

In [556]:
# Create the index
index = {}
for review in data:
    b_id = review['business_id']
    preprocessed_text = preprocess(review['text']) # Array of arrays of tokens
    bigrams = generate_bigrams(preprocessed_text)
    
    # index the unigrams
    for sentence in preprocessed_text:
        for word in sentence:
            try:
                index[word].add(b_id)
            except:
                index[word] = {b_id}
    # index the bigrams
    for bigram in bigrams:
        try:
            index[bigram].add(b_id)
        except:
            index[bigram] = {b_id}

In [557]:
# Since we don't need to list of business ids in the index, we convert it to the count of business ids
for key in index:
    index[key] = len(index[key])

In [558]:
index

{'mother': 56,
 'birthday': 72,
 'parti': 104,
 '102916': 1,
 'great': 153,
 'time': 153,
 'food': 136,
 'music': 90,
 'waiter': 64,
 'thank': 142,
 'lyle': 1,
 ('mother', 'birthday'): 1,
 ('birthday', 'parti'): 14,
 ('parti', '102916'): 1,
 ('great', 'time'): 44,
 ('food', 'music'): 3,
 ('music', 'waiter'): 1,
 ('waiter', 'great'): 5,
 ('thank', 'lyle'): 1,
 'good': 153,
 'korean': 17,
 'grill': 89,
 'near': 123,
 'eaton': 2,
 'centr': 10,
 'marin': 29,
 'got': 153,
 'beef': 89,
 'ox': 4,
 'liver': 8,
 'salmon': 41,
 'fish': 83,
 'fillet': 16,
 'chicken': 108,
 'pork': 56,
 'belli': 33,
 'bland': 86,
 'meh': 65,
 'realli': 153,
 'flavour': 39,
 'fun': 113,
 'place': 153,
 'eat': 136,
 'date': 93,
 'group': 111,
 'friend': 151,
 'even': 153,
 'alon': 85,
 'judgment': 11,
 'staff': 153,
 'attent': 126,
 'nice': 153,
 'consider': 37,
 'bigger': 91,
 'like': 153,
 'seat': 126,
 'second': 149,
 'floor': 88,
 'way': 153,
 'caution': 12,
 'smell': 97,
 'bbq': 54,
 ('good', 'korean'): 3,
 ('k

#### Calculating TF-IDF

###### Term Frequency
Term frequency is calculated by number of occurrences of the term (word or bigram) divided by the number of terms (word or bigram) in the entire document (in this case, the business)

In [559]:
# Find word/biword count stored in a dictionary { token : count }
token_count = {}
for review in data:
    if review['business_id'] == selected_business_id:
        # break up review text into sentences
        sentences = preprocess(review['text'])
        # find the count of each unigram
        for sentence in sentences:
            for word in sentence:
                try:
                    token_count[word] += 1
                except:
                    token_count[word] = 1
        # find the count of each bigram
        bigrams = generate_bigrams(sentences)
        for bigram in bigrams:
            try:
                token_count[bigram] += 1
            except:
                token_count[bigram] = 1

In [560]:
token_count

{'connor': 1,
 'nicest': 1,
 'guy': 3,
 'work': 19,
 'friendli': 38,
 'experi': 4,
 'explain': 4,
 'variou': 4,
 'yogurt': 127,
 'make': 20,
 'chill': 2,
 'environ': 2,
 ('connor', 'nicest'): 1,
 ('nicest', 'guy'): 1,
 ('guy', 'work'): 1,
 ('work', 'friendli'): 1,
 ('friendli', 'experi'): 1,
 ('experi', 'explain'): 1,
 ('explain', 'variou'): 1,
 ('variou', 'yogurt'): 1,
 ('make', 'chill'): 1,
 ('chill', 'environ'): 1,
 'great': 46,
 'locat': 8,
 'get': 42,
 'froyo': 14,
 'north': 4,
 'glendal': 6,
 '10': 4,
 'discount': 1,
 'midwestern': 1,
 'student': 1,
 'flavor': 66,
 'top': 85,
 'consist': 4,
 'good': 38,
 '-': 7,
 'problem': 2,
 'rotat': 1,
 'frequent': 5,
 'otherwis': 3,
 'clean': 44,
 'insid': 6,
 'employe': 20,
 'sampl': 30,
 'commit': 1,
 'particular': 1,
 'cup': 15,
 'also': 19,
 'reward': 3,
 'program': 4,
 'coupl': 5,
 'visit': 9,
 ('great', 'locat'): 1,
 ('locat', 'get'): 1,
 ('get', 'froyo'): 2,
 ('froyo', 'north'): 1,
 ('north', 'glendal'): 2,
 ('glendal', '10'): 1,
 ('1

In [561]:
# calculate the TF
def computeTF(ngram, count_dict):
    term_frequency = count_dict[ngram]
    # if ngram is a single word
    if type(ngram) == str:
        total_word_count = 0
        for i in count_dict:
            if type(i) == str:
                total_word_count += count_dict[i]
        return term_frequency / total_word_count
    # if ngram is a bigram
    elif type(ngram) == tuple:
        total_biword_count = 0
        for i in count_dict:
            if type(i) == tuple:
                total_biword_count += count_dict[i]
        return term_frequency / total_biword_count

###### Inverse Document Frequency
For the Inverse Document Frequency (IDF), we will be using a smoothed IDF, i.e. a constant value of 1 will be added to the numerator and denominator, so as to prevent zero divisions. IDF(t) = log ( (1+n) / (1+df(t)) ) + 1

In [562]:
# calculate IDF
def computeIDF(ngram, count_dict, N): # N is the total number of businesses
    df = count_dict[ngram]
    idf = math.log((1+N) / (1+df)) + 1
    return idf

###### TF-IDF
Now we are ready to calculate the TF-IDF. In order to calculate the TF-IDF of each adjective phrase, we first need to calculate the TF-IDF values of each individual unigram/bigram in the phrase, and sum up these values.

In [563]:
# compute TFIDF of a single token (word or biword)
def computeTFIDF(ngram, business_word_count_dict, total_word_count_dict, N):
    tf = computeTF(ngram, business_word_count_dict)
    idf = computeIDF(ngram, total_word_count_dict, N)
    return tf * idf

In [564]:
# compute TFIDF of an adjective phrase
def computePhraseTFIDF(adjective_phrase, business_word_count_dict, total_word_count_dict, N):
    sum_of_tfidf = 0
    tokens = []
    # preprocess the adjective phrase
    preprocessed_phrase = preprocess(adjective_phrase)
    
    # break down adjective phrase into bigrams if it is more than one word
    try:
        if len(preprocessed_phrase[0]) == 1:
            tokens = preprocessed_phrase[0]
        else:
            tokens = generate_bigrams(preprocessed_phrase)
    except:
        # phrase has no words
        return 0
    
    # Loop through tokens to calculate TFIDF and sum them up
    for token in tokens:
        sum_of_tfidf += computeTFIDF(token, business_word_count_dict, total_word_count_dict, N)
    
    return sum_of_tfidf

In [565]:
# calculate the tfidf of each adjective phrase
adjp_tfidf = {}

for adjp in adjective_phrases:
    adjp_tfidf[adjp] = computePhraseTFIDF(adjp, token_count, index, len(business_set))

In [566]:
adjp_tfidf

{'long': 0.0011968069928908365,
 'available': 0.0008796182337791007,
 'expensive but worth': 0.0013721423489151206,
 'slightly higher': 0.0010996140763361469,
 'wonderful': 0.0007814341354133115,
 'so sparkling clean and bright': 0.0035656413303724287,
 'least': 0.0014932509141044844,
 'other popular': 0.000811519268550439,
 'most competitive in yogurt': 0.0014848028401927434,
 'not ashamed': 0.0009131338941920119,
 'great after dinner': 0.001011130127706379,
 'big': 0.000718084195734502,
 'friendliest': 0.0007763053222767771,
 'infinite': 0.0010095584145506716,
 'always very nice': 0.000689526129737209,
 'little': 0.0035671819262782403,
 'sure': 0.0016646848989298453,
 'so make': 0.0047562425683709865,
 'great with money': 0.0013721423489151206,
 'few different': 0.0019148911886253386,
 'better': 0.002853745541022592,
 'single': 0.0006762578290475006,
 'vast and plentiful': 0.0014848028401927434,
 'happy': 0.0009636833118357695,
 'tangy': 0.0006608456752418899,
 'pretty darn reasonabl

In [567]:
# Convert the adjective phrases' TFIDF values into a pandas dataframe
adjp_key = []
tfidf_value = []
for key in adjp_tfidf:
    adjp_key.append(key)
    tfidf_value.append(adjp_tfidf[key])

adjp_tfidf_dict = {
    "Adjective Phrase": adjp_key,
    "TF-IDF": tfidf_value
}
    
adjp_tfidf_df = pd.DataFrame(adjp_tfidf_dict)

In [568]:
adjp_tfidf_df

,Adjective Phrase,TF-IDF
0,long,0.001197
1,available,0.000880
2,expensive but worth,0.001372
3,slightly higher,0.001100
4,wonderful,0.000781
...,...,...
400,much better,0.000748
401,too fake,0.000643
402,consistently great with every visit,0.003185
403,young,0.000376


In [569]:
fig = px.box(adjp_tfidf_df, x="TF-IDF", hover_data=['Adjective Phrase'], points="all", title="TF-IDF values for Adjective Phrases of Business ID " + selected_business_id)
fig.show()

In [570]:
adjp_tfidf_df.describe()

,TF-IDF
count,405.000000
mean,0.002145
std,0.002974
min,0.000000
25%,0.000792
50%,0.001325
75%,0.001896
max,0.020346


##### Indicative Adjective Phrases
Let's consider the indicative adjective phrases of the business b1 to be the phrases that have a TF-IDF value that is the greater than or equal to the upper fence of all the TF-IDF value. The upper fence can be calculated as 1.5 * IQR + Q3 (3rd Quartile + 1.5 x Interquartile Range).

In [571]:
# Find the upper fence value
q3 = adjp_tfidf_df.describe()['TF-IDF']["75%"]
q1 = adjp_tfidf_df.describe()['TF-IDF']["25%"]
iqr = q3 - q1
upper_fence = q3 + 1.5 * iqr
print("Upper fence TF-IDF: " + str(upper_fence))

Upper fence TF-IDF: 0.0035531796530631227


In [572]:
# Extract out the phrases that have tf-idf value >= upper fence
indicative_df = adjp_tfidf_df[adjp_tfidf_df["TF-IDF"] >= upper_fence]

In [573]:
# Sort in descending order
indicative_df = indicative_df.sort_values("TF-IDF", ascending=False)
indicative_df = indicative_df.reset_index(drop=True)

In [574]:
indicative_df

,Adjective Phrase,TF-IDF
0,topping,0.020346
1,oatmeal,0.019039
2,other froyo,0.017792
3,froyo,0.017792
4,only frozen,0.016389
5,frozen,0.016389
6,fro-yo,0.014378
7,other fro-yo,0.014378
8,sample,0.013288
9,n't been,0.013080
